## Algorithm:




```
S1: Initialize G to the set of maximally general hypotheses in H
S2: Initialize S to the set of maximally specific hypotheses in H
S3: For each training example d, do

    (A)  If d is a positive example
        *  Remove from G any hypothesis inconsistent with d
        *  For each hypothesis s in S that is not consistent with d,
            *  Remove s from S
            *  Add to S all minimal generalizations h of s such that
                *  h is consistent with d, and some member of G is more general than h
            *  Remove from S any hypothesis that is more general than another hypothesis in S

    (A)  If d is a negative example
        *  Remove from S any hypothesis inconsistent with d
        *  For each hypothesis g in G that is not consistent with d,
            *  Remove g from G
            *  Add to G all minimal specializations h of g such that
                *  h is consistent with d, and some member of S is more specific than h
            *  Remove from G any hypothesis that is less general than another hypothesis in G

S4: Output the hypothesis

```





# Code:

### *Import and initialization*

In [1]:
import pandas as pd
import numpy as np
import re

In [54]:
df = pd.read_csv("data.csv")

In [92]:
# testing with shuffling input and adding new row
df.loc[len(df.index)] = ['rainy','warm','normal','weak','warm','same','yes']

In [93]:
df = df.sample(frac = 1) 
df.head()

,sky,air temp,humidity,wind,water,forecast,enjoy sport
0,sunny,warm,normal,strong,warm,same,yes
1,sunny,warm,high,strong,warm,same,yes
4,rainy,warm,normal,weak,warm,same,yes
3,sunny,warm,high,strong,cool,change,yes
2,rainy,cold,high,strong,warm,change,no


In [94]:
d = df.iloc[:,:-1]
print("The attributes are: ",d.columns.values)

target = df.iloc[:,-1]
print("The target is: ",target.values)

The attributes are:  ['sky' 'air temp' 'humidity' 'wind' 'water' 'forecast']
The target is:  ['yes' 'yes' 'yes' 'yes' 'no']


### *Functions*

In [46]:
# to evaluate an instance h with feature s
def evaluate(x,s):
    for i, j in zip(x,s):
        if i != j and i != '?':
            return 'no'

    return 'yes'

In [47]:
# to remove the inconsistent instances from given G or S
def removeInconsistent(x,d1):
    flag = False
    for i in x:
        val = evaluate(i,d1)
        if val != 'yes':
            x.remove(i)
            break

In [48]:
# to perform the Computation
def performCE(x,d1,d,t1,s1,t):
    for i in x:
        if t1 == 'yes':
            x = generateS(x,x.index(i),d1)
    if t1 == 'no':
        if s1[0] == '0':
            s1 = preIni(d,t)
        temp = generateG(x[:],x.index(i),d[:d1+1],s1, t[:d1+1])
        x = temp
    return x

def preIni(d,t):
    for i, j in zip(d,t):
        if j == 'yes':
            return i


In [49]:
# to generate S instance
def generateS(x,i,s):
    for m, n in zip(range(len(x[i])), s):
        if x[i][m] != n:
            if x[i][m] == '0':
                x[i][m] = n
            else:
                x[i][m] = "?"


In [50]:
# to generate G instances
def generateG(x,i,d,s1,t):
    g = x.pop(i)
    for i, j in zip(range(len(g)),s1):
        if g[i] != j and g[i] =='?':
            temp = g[:]
            temp[i] = s1[i]
            flag = True
            if temp not in x:
                for m, n in zip(d,t):
                    if n=='no' and evaluate(temp,m) != n:
                        flag = False
                        break
            if flag:
                x.append(temp)
    return x

In [51]:
# main function
def candidateElimination(d, t):
    G = [['?' for i in range(len(d[0]))]]
    S = [['0' for i in range(len(d[0]))]]
    for i, j in zip(range(len(d)),t):
        if re.match(j,'yes',re.I):
            removeInconsistent(G,d[i])
            performCE(S,d[i],None,j,None,None)

        else:
            G = performCE(G,i,d,j,S[-1],t)
    print("G includes:")
    printThem(G)
    print("S includes:")
    printThem(S)

In [52]:
def printThem(x):
    print("-"*10)
    for i in x:
        print(i)
    print("\n")

### *Output*

In [95]:
candidateElimination(d.values,target.values)

G includes:
----------
['?', 'warm', '?', '?', '?', '?']


S includes:
----------
['?', 'warm', '?', '?', '?', '?']




## Version - 2

In [33]:
def generalizeS(d1,S):
    for i,j in enumerate(zip(d1,S[0])):
        if j[0] != j[1] and j[1] != '?':
            if j[1] != None:
                S[0][i] = '?'
            else:
                S[0][i] = j[0]

In [79]:
def specializeG(G,s1,d1):
    for i,j in enumerate(zip(G[0],s1,d1)):
        temp = G[0].copy()
        if j[0] != j[1] and j[0] == '?' and j[1] != j[2]:
            temp[i] = j[1]
            G.append(temp)
    G.pop(0)

In [90]:
def candidate_elimination(d,t):
    n = len(t)
    S = [[None for i in range(len(d[0]))]]
    G = [['?' for i in range(len(d[0]))]]
    neg = []
    pos = []
    for i in range(n):
        if t[i] == 'yes': pos.append(i)
        else: neg.append(i)

    # for positive instance
    for i in pos:
        generalizeS(d[i],S)
    # for positive instance
    for j in neg:
        specializeG(G,S[0],d[j])

    print("Specific Hypothesis: ", S[0], sep='\n')
    print("\nGeneral Hypothesis: ")
    for i in G:
        print(i)
    

In [96]:
candidate_elimination(d.values,target.values)

Specific Hypothesis: 
['?', 'warm', '?', '?', '?', '?']

General Hypothesis: 
['?', 'warm', '?', '?', '?', '?']
